Danyal Faheem
19I-2014 CS-F
Assignment 1

Importing our dataset here and creating relevant dictionaries

In [1]:
# Danyal Faheem
# 19I-2014 CS-F
# Assignment 1
# Reading Files here
import copy
file = open("data3.txt")
str = file.readline()
char = str[0]
parent = dict()
table = dict()
# Reading file line by line as we create dictionaries
while str:
    # Check if starting letter has changed
    if char == str[0]:
        # Tokenizing the line
        str = str.strip().split(",")
        # Checking if multiple parent sets avaible
        if len(str) > 1:
            if len(str) == 4:
                str = [str[0], ",".join([str[1], str[2]]),  str[3]]
            parent[str[1]] = str[2]
    else:
        # If letter has changed add all of the current set to one dictionary
        table[char] = copy.deepcopy(parent)
        # Clear the current set
        parent.clear()
        # Making sure that current read line is also included
        str = str.strip().split(",")
        if len(str) > 1:
            if len(str) == 4:
                str = [str[0], ",".join([str[1], str[2]]),  str[3]]
            parent[str[1]] = str[2]
    # Reiterate with next line
    char = str[0]
    str = file.readline()

# Adding last set to dictionary
table[char] = copy.deepcopy(parent)
file.close()

# Removing any extra empty lines that we might have read
delete = []
for x in table:
    if x == "":
        delete.append(x)

for y in delete:
    table.pop(y)

# Printing our dictionaries that we created
# for x in table:
#     print(x, " ->")
#     for y in table[x]:
#         print(y, ":", table[x][y])
# # print(table)


Cost Calculation Function for each ordering set

In [2]:
from itertools import permutations
# Our main function which takes the orderingSet as input and returns its cost based on the table created above
def calculateCost(orderingSet, printPath=False):
    cost = 0
    index = ""
    # Iterate over the set
    for i in range(len(orderingSet)):
        # If first index, meaning no parent
        if i == 0 and table[orderingSet[i]].get("{}") != None:
            index = "{}"
            cost += float(table[orderingSet[i]][index])
        # If not the first index
        else:
            check = False
            minCost = 100000000
            # Iterating over each parent index of the current index
            for y in range(i + 1):
                # Checking if double parents have a cost that is less
                if y > 1:
                    index = "{" + orderingSet[y - 2] + "," + orderingSet[y - 1] + "}"
                    if table[orderingSet[i]].get(index) != None and minCost > float(table[orderingSet[i]][index]):
                        minCost = float(table[orderingSet[i]][index])
                        check = True
                # Checking for each possible parent set
                index = "{" + orderingSet[y] + "}"
                if table[orderingSet[i]].get(index) != None and minCost > float(table[orderingSet[i]][index]):
                    minCost = float(table[orderingSet[i]][index])
                    check = True 
            # If no parent set found, then assign empty set   
            if check == False and table[orderingSet[i]].get("{}") != None:
                index = "{}"
                minCost = float(table[orderingSet[i]][index])
                cost += minCost
            # If empty set has less cost
            if table[orderingSet[i]].get("{}") != None and minCost > float(table[orderingSet[i]]["{}"]):
                index = "{}"
                cost += float(table[orderingSet[i]][index])
                check = False
            # Finally add the cost
            if check == True:
                cost += minCost
        # Printing path if parameter passed
        if printPath:
            print(index, end="->")
            print(cost)
    return cost

# Checking for all permutations using the built in permutations function
orderingSet = [x for x in table]
minSet = []
miniCost = 100000000

for set1 in permutations(orderingSet):
    # print(set, calculateCost(set))
    if miniCost > calculateCost(set1):
        miniCost = calculateCost(set1)
        minSet = set1

print(minSet, calculateCost(minSet, 1))

{}->169.482
{2}->291.058
{5}->327.566
{3}->369.34099999999995
{1}->465.43399999999997
('4', '2', '5', '3', '1') 465.43399999999997


Making our Graph and Node Class 

In [3]:
import numpy as np

class Node:
    def __init__(self, vertix):
        self.vertix = vertix
        self.edges = []

    def addEdge(self, edge):
        if edge not in self.edges:
            self.edges.append(edge)

    def removeEdge(self, edge):
        if edge in self.edges:
            self.edges.remove(edge)

    def printNode(self):
        print(np.matrix(self.vertix), "->",  self.edges)

    def __repr__(self):
        return str(self.vertix) + str(self.edges)

    def __str__(self):
        return str(self.vertix) + str(self.edges)

    def getVertix(self):
        return self.vertix

    def getEdges(self):
        return self.edges


class Graph:
    def __init__(self):
        self.noOfVertices = 0
        self.G = []

    def addNode(self, vertix):
        if vertix not in self.G:
            self.G.append(Node(vertix))
            self.noOfVertices += 1

    def printGraph(self):
        for i in self.G:
            print(i.printNode())

    def addEdge(self, src, dest):
        if self.findNode(src) != -1 and self.findNode(dest) != -1:
            self.G[self.findNode(src)].addEdge(dest)
            self.G[self.findNode(dest)].addEdge(src)

    def findNode(self, node):
        for nodes in self.G:
            if nodes.getVertix() == node:
                return self.G.index(nodes)
        return -1


Applying BFS to get minimum cost

In [4]:
# Breadth First Search on our current problem, takes no input parameters, prints the minimum cost, orderingSet and it's path taken
def BFS():
    # Intialize our variables
    visited = []
    minimumCost = 10000000
    minimumCostNode = []
    # orderingSet here is our in order read file possible vertices
    orderingSet = [x for x in table]
    queue = [tuple(orderingSet)]
    currentNode = tuple()
    # Iterate while there is something in queue
    while queue:
        # Using FIFO strategy, remove from front of queue
        currentNode = queue.pop(0)
        # Check if already visited, if not then perform operations
        if currentNode not in visited:
            visited.append(currentNode)
            # Checking for minimum Cost node
            if minimumCost > calculateCost(currentNode):
                minimumCost = calculateCost(currentNode)
                minimumCostNode = currentNode
            # Generate new nodes to add to queue
            for i in range(len(orderingSet)):
                currentNodeCopy = list(copy.deepcopy(currentNode))
                # Using a cyclic swapping strategy, iterate over all possible combinations
                currentNodeCopy[i], currentNodeCopy[(i + 1) % len(orderingSet)] = currentNodeCopy[(i + 1) % len(orderingSet)], currentNodeCopy[i]
                if tuple(currentNodeCopy) not in visited and tuple(currentNodeCopy) not in queue:
                    # Add to queue
                    queue.append(tuple(currentNodeCopy))
    # Print the minimum cost ordering set as well as it's cost and path taken by each parent
    print(minimumCostNode, calculateCost(minimumCostNode, 1))
BFS()

{}->169.802
{3}->211.577
{1}->307.67
{4}->343.858
{2}->465.43399999999997
('5', '3', '1', '4', '2') 465.43399999999997


Applying DFS to get minimum cost

In [5]:
# Depth First Search on our current problem, takes no input parameters, prints the minimum cost, orderingSet and it's path taken
def DFS():
    # Intialize our variables
    visited = []
    minimumCost = 10000000
    minimumCostNode = []
    # orderingSet here is our in order read file possible vertices
    orderingSet = [x for x in table]
    stack = [tuple(orderingSet)]
    currentNode = tuple()
    # Iterate while there is something in stack
    while stack:
        # Using LIFO strategy, remove from front of stack
        currentNode = stack.pop()
        # Check if already visited, if not then perform operations
        if currentNode not in visited:
            visited.append(currentNode)
            # Checking for minimum Cost node
            if minimumCost > calculateCost(currentNode):
                minimumCost = calculateCost(currentNode)
                minimumCostNode = currentNode
            # Generate new nodes to add to stack
            for i in range(len(orderingSet)):
                currentNodeCopy = list(copy.deepcopy(currentNode))
                # Using a cyclic swapping strategy, iterate over all possible combinations
                currentNodeCopy[i], currentNodeCopy[(i + 1) % len(orderingSet)] = currentNodeCopy[(i + 1) % len(orderingSet)], currentNodeCopy[i]
                if tuple(currentNodeCopy) not in visited and tuple(currentNodeCopy) not in stack:
                    # Add to stack
                    stack.append(tuple(currentNodeCopy))
    # Print the minimum cost ordering set as well as it's cost and path taken by each parent
    print(minimumCostNode, calculateCost(minimumCostNode, 1))
DFS()

{}->169.482
{2}->291.058
{5}->327.566
{3}->369.34099999999995
{1}->465.43399999999997
('4', '2', '5', '3', '1') 465.43399999999997


Applying UCS to get minimum cost

In [6]:
def sortFringe(fringe):
    for i in range(len(fringe)):
        check = False
        for j in range(i + 1, len(fringe) - 1):
            if calculateCost(fringe[j]) > calculateCost(fringe[j + 1]):
                fringe[j], fringe[j + 1] = fringe[j + 1], fringe[j]
                check = True
        if check == False:
            return fringe
    return fringe

# Uniform Cost Search on our current problem, takes no input parameters, prints the minimum cost, orderingSet and it's path taken
def UCS():
    # Intialize our variables
    visited = []
    minimumCost = 10000000
    minimumCostNode = []
    # orderingSet here is our in order read file possible vertices
    orderingSet = [x for x in table]
    fringe = [tuple(orderingSet)]
    currentNode = tuple()
    # Iterate while there is something in fringe
    while fringe:
        # Sort fringe to put the shortest cost path at the front of fringe
        fringe = sortFringe(fringe)
        # Using FIFO strategy, remove from front of fringe
        currentNode = fringe.pop(0)
        # Check if already visited, if not then perform operations
        if currentNode not in visited:
            visited.append(currentNode)
            # Checking for minimum Cost node
            if minimumCost > calculateCost(currentNode):
                minimumCost = calculateCost(currentNode)
                minimumCostNode = currentNode
            # Generate new nodes to add to fringe
            for i in range(len(orderingSet)):
                currentNodeCopy = list(copy.deepcopy(currentNode))
                # Using a cyclic swapping strategy, iterate over all possible combinations
                currentNodeCopy[i], currentNodeCopy[(i + 1) % len(orderingSet)] = currentNodeCopy[(i + 1) % len(orderingSet)], currentNodeCopy[i]
                if tuple(currentNodeCopy) not in visited and tuple(currentNodeCopy) not in fringe:
                    # Add to queue
                    fringe.append(tuple(currentNodeCopy))
    # Print the minimum cost ordering set as well as it's cost and path taken by each parent
    print(minimumCostNode, calculateCost(minimumCostNode, 1))
UCS()

{}->169.802
{3}->211.577
{1}->307.67
{4}->343.858
{2}->465.43399999999997
('5', '3', '1', '4', '2') 465.43399999999997
